In [2]:
!pip install transformers datasets accelerate peft rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f35b58c59ace63b267bb4b3e83da4cedabec7fa903820cd4d18794be81ab6e3d
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


# Step 1: Data Preparation

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the CNN/Daily Mail summarization dataset
# This is a well-maintained dataset that will load without issues.
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:10000]")

# Initialize the tokenizer for the T5 model
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    # This prepares the data for a sequence-to-sequence model like T5
    model_inputs = tokenizer(
        examples["article"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        examples["highlights"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    # T5 models expect the labels to be named 'labels'
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights", "id"])

# Split into training and validation sets
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

print("Data loaded and tokenized successfully. ✅")
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(eval_dataset)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Data loaded and tokenized successfully. ✅
Training dataset size: 9000
Validation dataset size: 1000


# Step 2: Model Fine-Tuning with PEFT

We'll fine-tune the model using the PEFT library.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

# Load the base T5 model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# --- PEFT Configuration (LoRA) ---
# LoRA (Low-Rank Adaptation) is a technique that only fine-tunes a small number
# of new, trainable parameters. This makes training very efficient.
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,  # Rank of the update matrices
    lora_alpha=32, # Scaling factor for LoRA
    lora_dropout=0.1,
    target_modules=["q", "v"] # The layers to apply LoRA to (query and value)
)

# Apply the PEFT configuration to the base model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# --- Training Arguments and Trainer ---
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1, # Start with a small number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none" # Disable integrations like WandB for simplicity
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Start training
trainer.train()

print("Model fine-tuning complete.")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


Step,Training Loss
10,13.193600
20,9.870300
30,6.674300
40,3.655900
50,2.071000
60,1.904100
70,1.673900
80,1.519300
90,1.432500
100,1.324900


Model fine-tuning complete.


# Step 3: Evaluation and Generation
After training, we will evaluate our model's performance and generate some new abstracts. We'll use the common ROUGE metric for evaluation.

In [ ]:
from rouge_score import rouge_scorer

# Function to generate an abstract and evaluate it
def generate_and_evaluate(model, tokenizer, dataset, num_examples=5):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []

    # Loop through a small number of examples from the validation dataset
    for i in range(num_examples):
        example = dataset[i]

        # Decode the tokenized input to get the original text
        input_text = tokenizer.decode(example["input_ids"], skip_special_tokens=True)
        reference_summary = tokenizer.decode(example["labels"], skip_special_tokens=True)

        # Generate a new abstract
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)
        # Fix: Pass input_ids as a keyword argument
        generated_ids = model.generate(input_ids=input_ids, max_length=128)
        generated_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # Calculate ROUGE score
        scores = scorer.score(reference_summary, generated_summary)
        rouge_scores.append(scores)

        print(f"--- Example {i+1} ---")
        print(f"Input Text (First 100 chars): {input_text[:100]}...")
        print(f"Reference Abstract: {reference_summary}")
        print(f"Generated Abstract: {generated_summary}")
        print("-" * 20)

    # Calculate average scores
    avg_scores = {
        key: sum(score[key].fmeasure for score in rouge_scores) / len(rouge_scores)
        for key in ['rouge1', 'rouge2', 'rougeL']
    }
    print("--- Average ROUGE Scores ---")
    print(avg_scores)

# Run the evaluation
generate_and_evaluate(model, tokenizer, eval_dataset)

--- Example 1 ---
Input Text (First 100 chars): ST. BERNARD PARISH, Louisiana (CNN) -- "We were a normal red-blooded American family," recalls Rudy ...
Reference Abstract: Katrina left St. Bernard Parish, once home to 67,000, virtually uninhabitable. Liz McCartney moved from Washington to New Orleans to help rebuild homes. She and her boyfriend started St. Bernard Project, focusing on seniors, families.
Generated Abstract: Rudy Aguilar lost everything during Hurricane Katrina in August 2005. Liz McCartney moved to St. Bernard Parish to help rebuilding homes. The nonprofit is dedicated to helping rebuilding homes, rebuilding homes.
--------------------
--- Example 2 ---
Input Text (First 100 chars): WASHINGTON (CNN) -- Expressing hope that the future of Tibet and China will move beyond mistrust, th...
Reference Abstract: NEW: Dalai Lama: China's allegation of a hidden agenda is unfounded, untrue. President Bush encourages China to sit down with "man of peace" White House says it isn't "

# Analyzing the results:
A "good" ROUGE score is highly dependent on the dataset and task. For abstractive summarization on the CNN/Daily Mail dataset, which is a challenging task, these scores are on the lower end of what a well-tuned model can achieve. For context, state-of-the-art models often achieve ROUGE-1 scores in the 0.40s and higher, and ROUGE-2 scores above 0.20. Our scores suggest that our model is capturing some of the key concepts (ROUGE-1) but struggles with capturing meaningful phrases and longer word sequences (ROUGE-2 and ROUGE-L).

# Next Steps: Improving the Model 🚀
Increase Training Epochs: We only trained for 1 epoch. Increase this to at least 3-5 epochs. This will give the model more time to learn the patterns in the data and reduce repetition.

Use a Larger Model: The t5-small model has only 60 million parameters. Switching to t5-base (220 million parameters) will significantly improve the model's capacity to learn complex relationships and generate more fluent text. All the code we've written will work with this larger model; just change the model_name variable.

Optimize Hyperparameters: Experiment with the learning_rate, batch_size, and lora_alpha to find a combination that produces better results.

Use a More Advanced Decoding Strategy: The default generate method uses a greedy search. We can improve output quality by using a different decoding strategy like Beam Search. This can be implemented by adding arguments to the model.generate() call in our evaluation function.

In [10]:
# --- REVISED STEP 1: Data Preparation ---
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the CNN/Daily Mail summarization dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:10000]")

# Split into training and validation sets before tokenization
raw_dataset = dataset.train_test_split(test_size=0.1)
train_raw_dataset = raw_dataset["train"]
eval_raw_dataset = raw_dataset["test"]

# Initialize the tokenizer for the T5 model
model_name = "t5-base" # Use the larger model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        examples["highlights"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the tokenization to the raw splits
tokenized_train_dataset = train_raw_dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights", "id"])
tokenized_eval_dataset = eval_raw_dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights", "id"])

print("Data loaded and tokenized successfully. ✅")
print(f"Training dataset size: {len(tokenized_train_dataset)}")
print(f"Validation dataset size: {len(tokenized_eval_dataset)}")



Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Data loaded and tokenized successfully. ✅
Training dataset size: 9000
Validation dataset size: 1000


## Step 1: Model Scaling and Configuration

Reasoning:

t5-base: This model has 220 million parameters, significantly more than t5-small's 60 million. This increase in capacity will allow it to learn more nuanced relationships and produce more coherent output.

LoRA r Parameter: We've increased the rank of the update matrices from 8 to 16. A higher rank allows the LoRA adapters to learn more complex patterns, leading to a better-performing model.

In [11]:
# Change the model name to use a larger T5 variant
model_name = "t5-base"

# Re-initialize the tokenizer and model with the new, larger model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# --- PEFT Configuration (LoRA) ---
# We'll use the same configuration, as it works well with t5-base
from peft import LoraConfig, get_peft_model, TaskType
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=16,  # Increase rank for a more expressive model
    lora_alpha=32, # Keep the scaling factor
    lora_dropout=0.1,
    target_modules=["q", "v"]
)

# Apply the PEFT configuration to the base model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 224,673,024 || trainable%: 0.7876


## Step 2: Training with More Epochs
Our previous run used only a single epoch, which is not enough for the model to learn the intricacies of summarization.

In [12]:
from transformers import TrainingArguments, Trainer

# --- Training Arguments and Trainer ---
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-4,
    per_device_train_batch_size=8, # Increase batch size if GPU allows
    per_device_eval_batch_size=8,
    num_train_epochs=3, # Increase epochs to allow for more learning
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Start training
trainer.train()

print("Model fine-tuning complete.")

Step,Training Loss
10,7.633600
20,5.349500
30,2.472000
40,1.446200
50,1.261700
60,1.182500
70,1.072100
80,0.997300
90,1.061700
100,1.010700


Model fine-tuning complete.


## Step 3: Advanced Generation Strategy
The default greedy search is a simple but naive way to generate text. It simply picks the most likely next token at each step. This can lead to the repetitive phrases we observed. Beam search is a more sophisticated method that explores multiple possible sequences.

In [13]:
# --- REVISED STEP 3: Evaluation and Generation ---
# In this step, we will pass the UNTOKENIZED eval_raw_dataset
# to the generate_and_evaluate function.

from rouge_score import rouge_scorer
import torch

def generate_and_evaluate(model, tokenizer, dataset, num_examples=5):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for i in range(num_examples):
        example = dataset[i]
        # These columns exist in the untokenized dataset
        input_text = example["article"]
        reference_summary = example["highlights"]

        # This will work now
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
        generated_ids = model.generate(
            input_ids=input_ids,
            max_length=128,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=2
        )
        generated_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        scores = scorer.score(reference_summary, generated_summary)
        rouge_scores.append(scores)

        print(f"--- Example {i+1} ---")
        print(f"Input Text (First 100 chars): {input_text[:100]}...")
        print(f"Reference Abstract: {reference_summary}")
        print(f"Generated Abstract: {generated_summary}")
        print("-" * 20)

    avg_scores = {
        key: sum(score[key].fmeasure for score in rouge_scores) / len(rouge_scores)
        for key in ['rouge1', 'rouge2', 'rougeL']
    }
    print("--- Average ROUGE Scores ---")
    print(avg_scores)

# Run the evaluation with the UNTOKENIZED validation dataset
generate_and_evaluate(model, tokenizer, eval_raw_dataset)

--- Example 1 ---
Input Text (First 100 chars): (CNN) -- Boeing is asking pilots who fly its 737 jets to pay careful attention to flight instruments...
Reference Abstract: Boeing urges 737 pilots to pay careful attention to flight instruments .
Dutch investigators said a faulty altimeter contributed to Netherlands crash .
Erroneous altitude data caused automatic throttles to slow the plane down .
The Turkish Airlines jet broke into three pieces on impact, killing nine people .
Generated Abstract: Dutch investigators said a faulty altimeter contributed to the crash . The plane broke into three pieces near Schiphol Airport on February 25, according to memo to pilots. Boeing memo says automatic pilot should not be used for landings if the plane's Altimeters are malfunctioning - Pilots didn't recognize the problem until it was too late, he said.
--------------------
--- Example 2 ---
Input Text (First 100 chars): (CNN) -- Suspected Al Qaeda militants disguised as security forces launched a

# Qualitative Analysis

Qualitative Analysis of Generated Abstracts
The generated abstracts show a notable improvement in fluency and coherence compared to the previous run. The model is now generating full, grammatically correct sentences and has significantly reduced the repetitive phrases we observed before.

Example 1: The generated abstract is very coherent and provides a clear summary of the event. It correctly identifies the key entities (Boeing, Dutch investigators, altimeter) and synthesizes information into a readable paragraph. The detail about the crash in February and the note about the autopilot shows a good grasp of the source text.

Example 2: The model correctly identifies the attack on the U.S. Embassy in Sanaa and mentions key details like the "massive fireball" and the attackers being in police uniforms. The mention of "Ramadan" and "Osama bin Laden" are also accurate details from the source article.

Example 3: This abstract is very concise and accurate. It captures the core points about the "divided opinion" and the "trending topics on Twitter" very well.

Examples 4 & 5: These show a good ability to extract key facts and present them in a logical way, although they are a bit more list-like than the others. The model identifies the main themes (cholesterol, heart disease, American Idol, Adam Lambert) and provides relevant details.

The use of Beam Search and no_repeat_ngram_size has been highly effective in combating the repetition issue.